# This notebook shows how to use Orchestrator APIs for user experiments

## Configure the Environment

In [14]:
import os

os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

os.environ['FABRIC_BASTION_USERNAME']='pruth'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/.ssh/id_rsa_fabric'

os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'
os.environ['FABRIC_BASTION_HOST_PRIVATE_IPV4'] = '192.168.11.226'
os.environ['FABRIC_BASTION_HOST_PRIVATE_IPV6'] = '2600:2701:5000:a902::c'

## Setup the Experiment

#### Import FABRIC API

In [15]:
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import fablib

## Create Slice
In Release 1.0, user is expected to assign the IP addresses manually. Please use the example comands indicated below:

## Configure Slice Parameters



In [16]:
slice_name = 'MySliceL2Bridge2'
site = 'TACC'
node1_name = 'Node1'
node2_name = 'Node2'
network_name='net1'
node1_nic_name = 'nic1'
node2_nic_name = 'nic2'
username = 'centos'
image = 'default_centos_8'
image_type = 'qcow2'
cores = 2
ram = 8
disk = 10

In [17]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)

    # Node1
    node1 = slice.add_node(name=node1_name, site=site)
    node1.set_capacities(cores=cores, ram=ram, disk=disk)
    node1.set_image(image)
    iface1 = node1.add_component(model='NIC_ConnectX_5', name=node1_nic_name).get_interfaces()[0]
    
    # Node2
    node2 = slice.add_node(name=node2_name, site=site)
    node2.set_capacities(cores=cores, ram=ram, disk=disk)
    node2.set_image(image)
    iface2 = node2.add_component(model='NIC_ConnectX_5', name=node2_nic_name).get_interfaces()[0]
    
    # Network
    net1 = slice.add_l2network(name=network_name, interfaces=[iface1, iface2])

    #Submit Slice Request
    slice.submit(wait_progress=True)
except Exception as e:
    print(f"Slice Fail: {e}")
    traceback.print_exc()

Waiting for slice ................... Slice state: StableOK


In [18]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"Slice: {slice.get_name()}")
    
    slice.wait(progress=True)
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

Slice: MySliceL2Bridge2
Waiting for slice . Slice state: StableOK


## Get the Slice

In [19]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"Slice: {slice.get_name()}")
except Exception as e:
    print(f"Get Slices Fail: {e}")
    traceback.print_exc()

Slice: MySliceL2Bridge2


## Get the Nodes

Retrieve the node information and save the management IP address.


### Configure Node1

Use ssh to configure eth1 on  node 1.  

```
ip addr add 192.168.10.51/24 dev eth1
```

In [20]:
try:
    node1 = slice.get_node(node1_name)
    
    stdout, stderr = node1.execute('sudo ip addr add 192.168.10.51/24 dev eth1')
    print (stdout)

    stdout, stderr = node1.execute('sudo ip addr list eth1')
    print (stdout)
    
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()



3: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether b8:ce:f6:19:09:46 brd ff:ff:ff:ff:ff:ff
    inet 192.168.10.51/24 scope global eth1
       valid_lft forever preferred_lft forever
    inet6 fe80::5d1b:f07:2007:858f/64 scope link noprefixroute 
       valid_lft forever preferred_lft forever



### Configure Node2

Use ssh to configure eth1 on each Node 2.  

```
ip addr add 192.168.10.52/24 dev eth1
```

In [21]:
try:
    node2 = slice.get_node(node2_name)
    
    stdout, stderr = node2.execute('sudo ip addr add 192.168.10.52/24 dev eth1')
    print (stdout)

    stdout, stderr = node2.execute('sudo ip addr list eth1')
    print (stdout)
    
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()


3: eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether b8:ce:f6:18:fd:32 brd ff:ff:ff:ff:ff:ff
    inet 192.168.10.52/24 scope global eth1
       valid_lft forever preferred_lft forever
    inet6 fe80::fc11:1e93:88c:ef04/64 scope link noprefixroute 
       valid_lft forever preferred_lft forever



## Delete Slice

In [22]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()